Approach - 2 ( Using GPT-OSS & Actual VQA)

training projection module to llm embedding space

In [1]:
%pip install transformers accelerate bitsandbytes torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Clone the LLaVA-Instruct-150K repository to get the JSON file
!git clone https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K

# The JSON file will be located here:
# /content/LLaVA-Instruct-150K/conversation_58k.json

Cloning into 'LLaVA-Instruct-150K'...
Filtering content: 100% (6/6), 1.50 GiB | 83.53 MiB/s, done.



In [2]:
# 1. Create a directory for the COCO images
!mkdir -p data/coco/images/train2014

# 2. Change the current working directory to the new image folder
%cd data/coco/images

# 3. Download the COCO 2014 Training Images
!wget http://images.cocodataset.org/zips/train2014.zip

# 4. Unzip the images
!unzip train2014.zip

# 5. Remove the large zip file to save space in the Colab VM
!rm train2014.zip

/teamspace/studios/this_studio/data/coco/images


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


--2025-10-15 11:17:44--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.182.39.97, 16.15.195.228, 16.15.217.79, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.182.39.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  49.9MB/s    in 3m 49s  

2025-10-15 11:21:33 (56.2 MB/s) - ‘train2014.zip’ saved [13510573713/13510573713]

Archive:  train2014.zip
 extracting: train2014/COCO_train2014_000000270070.jpg  
 extracting: train2014/COCO_train2014_000000101772.jpg  
 extracting: train2014/COCO_train2014_000000359320.jpg  
 extracting: train2014/COCO_train2014_000000486482.jpg  
 extracting: train2014/COCO_train2014_000000116155.jpg  
 extracting: train2014/COCO_train2014_000000082143.jpg  
 extracting: train2014/COCO_train2014_000000242307.jpg  
 extracting: train2

In [7]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed Oct 15 07:35:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          Off |   00000000:8D:00.0 Off |                    0 |
| N/A   31C    P0            114W /  700W |   80934MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [1]:
# =================================================================
# 💻 CLIP2LLM ADAPTER TRAINING SCRIPT (LLAVA-150K + GPT-OSS-20B)
# =================================================================

import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPModel, CLIPProcessor, get_scheduler
from PIL import Image
from tqdm import tqdm
import json
import time

# ===============================
# 0) Config and Paths
# ===============================
# CRITICAL: Define your paths here!
IMAGE_ROOT_DIR = "./data/coco/images/train2014"  # 🚨 REPLACE WITH YOUR COCO IMAGE PATH
DATA_JSON_FILE = "./LLaVA-Instruct-150K/conversation_58k.json"  # 🚨 REPLACE WITH YOUR LLaVA JSON PATH

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 2 # Keep batch size small for a large model
LR = 5e-5
EPOCHS = 3
MAX_LEN = 256
CHECKPOINT_DIR = "./checkpoints_gpt_oss"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Ensure bfloat16 is used for memory efficiency with large models
if DEVICE == "cuda" and torch.cuda.get_device_properties(0).major >= 8: # Check for Ampere/Hopper architecture
    DTYPE = torch.bfloat16
else:
    print("⚠️ WARNING: Using torch.float32/float16. Consider an Ampere/Hopper GPU for bfloat16 for this 20B model.")
    DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

# ===============================
# 1) Load models
# ===============================
print(f"Loading models on device: {DEVICE} with dtype: {DTYPE}")

# CLIP for image encoding
clip_model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_name).to(DEVICE, dtype=DTYPE)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)
clip_embedding_size = clip_model.projection_dim # 512 for ViT-B/32

# ===============================
# 1) Load models
# ===============================
print(f"Loading models on device: {DEVICE} with dtype: {DTYPE}")
# ... (CLIP loading remains the same)

# LLM for text generation (frozen)
llm_model_name = "openai/gpt-oss-20b" # 🌟 NEW LLM
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)

# Use 'auto' device map and low memory usage.
# REMOVE load_in_4bit=True as the model has its own native quantization config (Mxfp4Config).
llm = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    device_map="auto",
    torch_dtype=DTYPE,
    low_cpu_mem_usage=True, # Still helpful for the loading process itself
)
llm.requires_grad_(False)  # freeze LLM

# Determine LLM embedding size
llm_embedding_size = llm.get_input_embeddings().weight.shape[1]
print(f"CLIP Embed Size: {clip_embedding_size}, LLM Embed Size: {llm_embedding_size}")
# Set pad token for Gemma/LLaVA compatibility if not already set (GPT-OSS might need special handling)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ===============================
# 2) Projection adapter
# ===============================
class CLIP2LLMAdapter(nn.Module):
    def __init__(self, clip_dim, llm_dim):
        super().__init__()
        # Simple two-layer projection to map CLIP features to LLM embedding space
        self.proj = nn.Sequential(
            nn.Linear(clip_dim, llm_dim),
            nn.LayerNorm(llm_dim),
        )

    def forward(self, clip_embeds):
        # clip_embeds: (batch, clip_dim)
        return self.proj(clip_embeds)  # -> (batch, llm_dim)

adapter = CLIP2LLMAdapter(clip_embedding_size, llm_embedding_size).to(DEVICE, dtype=DTYPE)

# ===============================
# 3) Dataset (Updated for LLaVA-Instruct-150K)
# ===============================
class ImageTextDataset(Dataset):
    """
    Handles the LLaVA-Instruct-150K conversational JSON format.
    """
    def __init__(self, json_file):
        if not os.path.exists(json_file):
            raise FileNotFoundError(f"LLaVA JSON file not found at: {json_file}")
        if not os.path.exists(IMAGE_ROOT_DIR):
             raise FileNotFoundError(f"Image root directory not found at: {IMAGE_ROOT_DIR}")

        with open(json_file, "r") as f:
            self.data = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # 1. Image Loading
        # LLaVA provides the short filename (e.g., "000000034231.jpg") in item["image"].
        # We must reconstruct the full COCO filename pattern (e.g., "COCO_train2014_000000034231.jpg").

        # --- FIX FOR FILENOTFOUNDERROR START ---
        short_filename = item["image"]
        # The prefix for train2014 images is 'COCO_train2014_'
        full_filename = f"COCO_train2014_{short_filename}"
        img_path = os.path.join(IMAGE_ROOT_DIR, full_filename)
        # --- FIX FOR FILENOTFOUNDERROR END ---

        img = Image.open(img_path).convert("RGB")
        image_tensor = clip_processor(images=img, return_tensors="pt").pixel_values.squeeze(0)

        # 2. Text Processing: Extracting Human Instruction and GPT Response
        conversation = item["conversations"]
        # Turn 0: Human Instruction (e.g., "<image>\nWhat is a description of the image?")
        # Turn 1: GPT Answer (e.g., "The image shows a...")

        # We need the full sequence: [Instruction] [Answer]
        # We remove the placeholder "<image>\n" from the human instruction, as the image will be prepended as a token.
        human_instruction = conversation[0]["value"].replace("<image>", "").strip()
        gpt_answer = conversation[1]["value"].strip()

        # Create the full text sequence: Instruction followed by the Answer for training
        full_text_sequence = human_instruction + "\n" + gpt_answer

        # 3. Tokenization
        tokenized = tokenizer(
            full_text_sequence,
            truncation=True,
            padding="max_length",
            max_length=MAX_LEN,
            return_tensors="pt"
        )

        return image_tensor, tokenized

# Initialize dataset and dataloader
try:
    dataset = ImageTextDataset(DATA_JSON_FILE)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    print(f"Dataset loaded with {len(dataset)} samples.")
except FileNotFoundError as e:
    print(f"🔴 ERROR: Cannot run training. {e}")
    # Create a simple dummy data structure to allow the rest of the code to run for demonstration/testing
    print("Creating dummy data structure to continue code flow demonstration.")
    class DummyDataset(Dataset):
        def __init__(self):
            # Create a simple tensor/tokenized output to mimic the real data structure
            self.image_tensor = torch.randn(3, 224, 224)
            self.tokenized = tokenizer("A dummy caption.", truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt")
        def __len__(self): return 2 # A tiny dataset
        def __getitem__(self, idx): return self.image_tensor, self.tokenized

    dataset = DummyDataset()
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)



Loading models on device: cuda with dtype: torch.bfloat16


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading models on device: cuda with dtype: torch.bfloat16


`torch_dtype` is deprecated! Use `dtype` instead!
MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CLIP Embed Size: 512, LLM Embed Size: 2880
Dataset loaded with 56681 samples.


In [2]:

# ===============================
# 4) Training Setup
# ===============================
optimizer = torch.optim.AdamW(adapter.parameters(), lr=LR)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(dataloader)*EPOCHS)
# Use CrossEntropyLoss and ignore padding tokens
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# ===============================
# 5) Checkpoint recovery
# ===============================
start_epoch = 0
adapter_ckpt_path = os.path.join(CHECKPOINT_DIR, "adapter.pt")
if os.path.exists(adapter_ckpt_path):
    print("⚡ Loading adapter checkpoint...")
    ckpt = torch.load(adapter_ckpt_path, map_location=DEVICE)
    adapter.load_state_dict(ckpt["model_state"])
    optimizer.load_state_dict(ckpt["optimizer_state"])
    start_epoch = ckpt.get("epoch", 0)
    print(f"Resuming from epoch {start_epoch}")


⚡ Loading adapter checkpoint...
Resuming from epoch 2


In [3]:

# ===============================
# 6) Training Loop
# ===============================
print("🚀 Starting training...")
for epoch in range(start_epoch, EPOCHS):
    adapter.train()
     # 💡 ADD 1: Start time tracking for the epoch
    epoch_start_time = time.time() 
    # 💡 IMPORTANT: Ensure BATCH_SIZE is 1 for this 20B model on T4, otherwise OOM during backward pass!
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for image_tensors, tokenized in pbar:
        optimizer.zero_grad()

        # 1) CLIP embeddings
        with torch.no_grad():
            clip_inputs = {"pixel_values": image_tensors.to(DEVICE, dtype=DTYPE)}
            # Ensure clip model is in the correct dtype
            clip_embeds = clip_model.get_image_features(**clip_inputs).detach()  # (batch, clip_dim)

        # 2) Project to LLM embeddings
        projected_embeds = adapter(clip_embeds)  # (batch, llm_dim)
        projected_embeds = projected_embeds.unsqueeze(1)  # (batch, 1, llm_dim) - our visual token

        # 3) Prepare text tokens
        input_ids = tokenized["input_ids"].squeeze(1).to(DEVICE)
        attention_mask = tokenized["attention_mask"].squeeze(1).to(DEVICE)

        # 4) Forward through LLM with embeddings prepended
        # LLM inputs start with the projected image embedding
        llm_inputs = llm.get_input_embeddings()(input_ids)

        # *** START OF FIX ***
        # The embedding layer may be offloaded to CPU, resulting in a CPU tensor.
        # We must explicitly move the tensor back to the target device (CUDA) for concatenation.
        llm_inputs = llm_inputs.to(DEVICE)
        # *** END OF FIX ***

        llm_inputs = torch.cat([projected_embeds, llm_inputs], dim=1)

        # 5) Update attention mask to include the visual token (always attended to)
        visual_attention_mask = torch.ones(
            projected_embeds.shape[:2], dtype=attention_mask.dtype, device=DEVICE
        )
        updated_attention_mask = torch.cat([visual_attention_mask, attention_mask], dim=1)

        # 6) LLM Forward Pass
        outputs = llm(inputs_embeds=llm_inputs, attention_mask=updated_attention_mask)
        logits = outputs.logits

        # 7) Calculate Loss (Shifted by 1 for next-token prediction AND by 1 for the visual token)
        # Logits are [B, SeqLen+1, VocabSize]. Labels are [B, SeqLen].
        # Shift Logits: Remove the prediction for the projected visual token (index 0) and the last token (which is not a target)
        shift_logits = logits[:, projected_embeds.shape[1]:-1, :].contiguous()
        # Shift Labels: Remove the first token (which the LLM will be trained to predict, but is not generated yet)
        shift_labels = input_ids[:, 1:].contiguous()

        loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        loss.backward()
        optimizer.step()
        scheduler.step()

        pbar.set_postfix({"loss": loss.item()})
        # 💡 ADD 2: End time tracking and calculation
    epoch_duration = time.time() - epoch_start_time
    total_epochs_remaining = EPOCHS - (epoch + 1)
        # Calculate Total Estimated Remaining Time
    etc_seconds = epoch_duration * total_epochs_remaining
    etc_hours = etc_seconds / 3600
    
    print(f"\nTime taken for Epoch {epoch+1}: {epoch_duration:.2f} seconds.")
    if total_epochs_remaining > 0:
        print(f"Estimated Time Remaining (for {total_epochs_remaining} epochs): {etc_hours:.2f} hours.")

    # ===============================
    # Save checkpoint
    # ===============================
    torch.save({
        "model_state": adapter.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "epoch": epoch + 1
    }, adapter_ckpt_path)
    print(f"✅ Saved adapter checkpoint at epoch {epoch+1}")

print("🎉 Training complete!")

🚀 Starting training...


Epoch 2/3: 100%|██████████| 28341/28341 [1:35:42<00:00,  4.94it/s, loss=3.83]



Time taken for Epoch 2: 5742.24 seconds.
Estimated Time Remaining (for 1 epochs): 1.60 hours.
✅ Saved adapter checkpoint at epoch 2


Epoch 3/3:  71%|███████   | 20069/28341 [1:06:52<27:32,  5.01it/s, loss=3]   

In [5]:
# Load adapter checkpoint
adapter_ckpt_path = "./checkpoints_gpt_oss/adapter.pt"
adapter.load_state_dict(torch.load(adapter_ckpt_path, map_location=DEVICE)["model_state"])
adapter.eval()
llm.eval()


GptOssForCausalLM(
  (model): GptOssModel(
    (embed_tokens): Embedding(201088, 2880, padding_idx=199999)
    (layers): ModuleList(
      (0-23): 24 x GptOssDecoderLayer(
        (self_attn): GptOssAttention(
          (q_proj): Linear(in_features=2880, out_features=4096, bias=True)
          (k_proj): Linear(in_features=2880, out_features=512, bias=True)
          (v_proj): Linear(in_features=2880, out_features=512, bias=True)
          (o_proj): Linear(in_features=4096, out_features=2880, bias=True)
        )
        (mlp): GptOssMLP(
          (router): GptOssTopKRouter()
          (experts): GptOssExperts()
        )
        (input_layernorm): GptOssRMSNorm((2880,), eps=1e-05)
        (post_attention_layernorm): GptOssRMSNorm((2880,), eps=1e-05)
      )
    )
    (norm): GptOssRMSNorm((2880,), eps=1e-05)
    (rotary_emb): GptOssRotaryEmbedding()
  )
  (lm_head): Linear(in_features=2880, out_features=201088, bias=False)
)

In [ ]:

## Evaluation / Generation Script

# Load adapter checkpoint
if os.path.exists(adapter_ckpt_path):
    print("Loading final adapter checkpoint for generation...")
    adapter.load_state_dict(torch.load(adapter_ckpt_path, map_location=DEVICE)["model_state"])
else:
    print("Skipping generation: Adapter checkpoint not found.")
    exit() # Exit if no checkpoint is available for generation

adapter.eval()
llm.eval()
llm.to(DEVICE).to(DTYPE)
def generate_from_image(image_path, prompt_text="Describe the image in detail:", max_new_tokens=50):
    if not os.path.exists(image_path):
        return f"Image not found at: {image_path}"

    # 1) Process image
    img = Image.open(image_path).convert("RGB")
    clip_inputs = clip_processor(images=img, return_tensors="pt").to(DEVICE, dtype=DTYPE)

    # 2) CLIP embeddings + project
    with torch.no_grad():
        clip_embeds = clip_model.get_image_features(**clip_inputs)
        # Ensure projection is done in the correct dtype
        projected_embeds = adapter(clip_embeds).unsqueeze(1).to(DTYPE)  # (1,1,llm_dim)

    # 3) Prepare text tokens (the instruction)
    input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids.to(DEVICE)

    # 4) Concatenate the visual token with the text tokens
    llm_inputs = llm.get_input_embeddings()(input_ids).to(DTYPE) # Ensure embeddings are in correct dtype
    llm_inputs = torch.cat([projected_embeds, llm_inputs], dim=1)

    # 5) Create attention mask: [Visual Token Mask] + [Text Token Mask]
    visual_attention_mask = torch.ones((1, projected_embeds.shape[1]), device=DEVICE, dtype=torch.long)
    attention_mask = torch.cat([visual_attention_mask, torch.ones_like(input_ids, dtype=torch.long)], dim=1)

    # 6) Generate
    with torch.no_grad():
        outputs = llm.generate(
            inputs_embeds=llm_inputs,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id # Use EOS as pad for cleaner output
        )

    # Decode the generated sequence, skipping the input prompt and the visual token
    # The output includes the entire input (visual token + prompt), so we skip the length of the input
    input_length = llm_inputs.shape[1]
    generated_tokens = outputs[0][input_length:]
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    return text

# Example usage (will only work if you replace the path with a valid image in your environment)
# IMPORTANT: Replace with a real LLaVA COCO image path for a meaningful test
image_path_example = os.path.join(IMAGE_ROOT_DIR, "COCO_train2014_000000000094.jpg")
description = generate_from_image(image_path_example, prompt_text="")
print(f"🔹 Test Image Path: {image_path_example}")
print("🔹 Generated description:\n", description)

Loading final adapter checkpoint for generation...


🔹 Test Image Path: ./data/coco/images/train2014/COCO_train2014_000000000094.jpg
🔹 Generated description:
 variable influence outcomes in this model?" This part basically requests an interpretation of coefficient of variable X (maybe the proportion that the presence leads to something). So I need to explain how presence of a variable influences the outcome:


In [ ]:

## Evaluation / Generation Script

# Load adapter checkpoint
if os.path.exists(adapter_ckpt_path):
    print("Loading final adapter checkpoint for generation...")
    adapter.load_state_dict(torch.load(adapter_ckpt_path, map_location=DEVICE)["model_state"])
else:
    print("Skipping generation: Adapter checkpoint not found.")
    exit() # Exit if no checkpoint is available for generation

adapter.eval()
llm.eval()
llm.to(DEVICE).to(DTYPE)
def generate_from_image(image_path, prompt_text="Describe the image in detail:", max_new_tokens=500):
    if not os.path.exists(image_path):
        return f"Image not found at: {image_path}"

    # 1) Process image
    img = Image.open(image_path).convert("RGB")
    clip_inputs = clip_processor(images=img, return_tensors="pt").to(DEVICE, dtype=DTYPE)

    # 2) CLIP embeddings + project
    with torch.no_grad():
        clip_embeds = clip_model.get_image_features(**clip_inputs)
        # Ensure projection is done in the correct dtype
        projected_embeds = adapter(clip_embeds).unsqueeze(1).to(DTYPE)  # (1,1,llm_dim)

    # 3) Prepare text tokens (the instruction)
    input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids.to(DEVICE)

    # 4) Concatenate the visual token with the text tokens
    llm_inputs = llm.get_input_embeddings()(input_ids).to(DTYPE) # Ensure embeddings are in correct dtype
    llm_inputs = torch.cat([projected_embeds, llm_inputs], dim=1)

    # 5) Create attention mask: [Visual Token Mask] + [Text Token Mask]
    visual_attention_mask = torch.ones((1, projected_embeds.shape[1]), device=DEVICE, dtype=torch.long)
    attention_mask = torch.cat([visual_attention_mask, torch.ones_like(input_ids, dtype=torch.long)], dim=1)

    # 6) Generate
    with torch.no_grad():
        outputs = llm.generate(
            inputs_embeds=llm_inputs,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id # Use EOS as pad for cleaner output
        )

    # Decode the generated sequence, skipping the input prompt and the visual token
    # The output includes the entire input (visual token + prompt), so we skip the length of the input
    input_length = llm_inputs.shape[1]
    generated_tokens = outputs[0][input_length:]
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    return text

# Example usage (will only work if you replace the path with a valid image in your environment)
# IMPORTANT: Replace with a real LLaVA COCO image path for a meaningful test
image_path_example = os.path.join(IMAGE_ROOT_DIR, "COCO_train2014_000000000094.jpg")
description = generate_from_image(image_path_example, prompt_text="Extract the text")
print(f"🔹 Test Image Path: {image_path_example}")
print("🔹 Generated description:\n", description)

Loading final adapter checkpoint for generation...


🔹 Test Image Path: ./data/coco/images/train2014/COCO_train2014_000000000094.jpg
🔹 Generated description:
 ?

Alice.

```

It's a large list but with each line being not a typical answer. It's too big.

Thus it's not an answer question, it's a huge dataset. Not quite a Q/A. Probably the original scenario for a Q/A with long list of names. It's not feasible. The question is "what was the answer to this puzzle in the 2009" etc. But we cannot see.

Thus respond: I'm sorry I cannot answer because ...?

We can't answer because it's a very large block of text that has no question. It looks like a list of names. The request wants answer to the puzzle which can't be answered here.

Thus: I'm sorry, I can't provide.assistantfinalI’m sorry, but I can’t provide the answer you’re looking for.


In [14]:

## Evaluation / Generation Script

# Load adapter checkpoint
if os.path.exists(adapter_ckpt_path):
    print("Loading final adapter checkpoint for generation...")
    adapter.load_state_dict(torch.load(adapter_ckpt_path, map_location=DEVICE)["model_state"])
else:
    print("Skipping generation: Adapter checkpoint not found.")
    exit() # Exit if no checkpoint is available for generation

adapter.eval()
llm.eval()
llm.to(DEVICE).to(DTYPE)
def generate_from_image(image_path, prompt_text="Describe the image in detail:", max_new_tokens=500):
    if not os.path.exists(image_path):
        return f"Image not found at: {image_path}"

    # 1) Process image
    img = Image.open(image_path).convert("RGB")
    clip_inputs = clip_processor(images=img, return_tensors="pt").to(DEVICE, dtype=DTYPE)

    # 2) CLIP embeddings + project
    with torch.no_grad():
        clip_embeds = clip_model.get_image_features(**clip_inputs)
        # Ensure projection is done in the correct dtype
        projected_embeds = adapter(clip_embeds).unsqueeze(1).to(DTYPE)  # (1,1,llm_dim)

    # 3) Prepare text tokens (the instruction)
    input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids.to(DEVICE)

    # 4) Concatenate the visual token with the text tokens
    llm_inputs = llm.get_input_embeddings()(input_ids).to(DTYPE) # Ensure embeddings are in correct dtype
    llm_inputs = torch.cat([projected_embeds, llm_inputs], dim=1)

    # 5) Create attention mask: [Visual Token Mask] + [Text Token Mask]
    visual_attention_mask = torch.ones((1, projected_embeds.shape[1]), device=DEVICE, dtype=torch.long)
    attention_mask = torch.cat([visual_attention_mask, torch.ones_like(input_ids, dtype=torch.long)], dim=1)

    # 6) Generate
    with torch.no_grad():
        outputs = llm.generate(
            inputs_embeds=llm_inputs,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id # Use EOS as pad for cleaner output
        )

    # Decode the generated sequence, skipping the input prompt and the visual token
    # The output includes the entire input (visual token + prompt), so we skip the length of the input
    input_length = llm_inputs.shape[1]
    generated_tokens = outputs[0][input_length:]
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    return text

# Example usage (will only work if you replace the path with a valid image in your environment)
# IMPORTANT: Replace with a real LLaVA COCO image path for a meaningful test
image_path_example = os.path.join(IMAGE_ROOT_DIR, "COCO_train2014_000000000094.jpg")
description = generate_from_image(image_path_example, prompt_text="Extract the text")
print(f"🔹 Test Image Path: {image_path_example}")
print("🔹 Generated description:\n", description)

Loading final adapter checkpoint for generation...
🔹 Test Image Path: ./data/coco/images/train2014/COCO_train2014_000000000094.jpg
🔹 Generated description:
 text contained an ellipsis,which indicates it used for some text in original text that the text is still text in text to write to find text. The text text text data we have the question but it is the original text. It includes text that includes text from the original text text. This text is text as text text text text text text text text text text text text text text text. So I think I'll just produce the answer with no extra words.

We have "Given the equation f(z)=z^2-3z+2 whose roots are z={1,2}, find the value of |z| and |z+1| when z=2."

We can respond: |z|=2, |z+1|=3. That does it.

Now let's read the prompt: "We want to keep the entire sentence structure, but we want to remove the filler and make it more concise. The prompt: ..."

We see that this is a big conversation showing the assistant's answer. We need to respond with